In [ ]:
### Read in the data
img_data_dir = "data/imgs/" # replace with your own dir if necessary
img_label_file = "data/labels.csv"

## Given an image name, extracts the label from the labels.csv file. True if 1 False if -1
def extract_label(file_name):
    split = file_name.split("-") # split image name
    fall_name = split[0] + "-" + split[1] # reconstruct fall name
    frame = int(split[-1].split(".")[0]) # end of split is like 113.png. removes .png part
    fall_labels = labels_csv.loc[fall_name]
    frame_label = fall_labels.loc[fall_labels["frame"] == frame]
    return int(frame_label.label) > 0

### Loads in the data from images and stores it in an array. Order is important because 
### The labels are in the same order as the images are appended to the array
def load_img_data():
    imgs = []
    labels = []
    labels_csv = pd.read_csv(img_label_file)
    
    # Iterates through fall folders and the frames within each fall
    for img_folder in os.listdir(img_data_dir):
        path = os.path.join(img_data_dir,img_folder)
        if ("fall" in path):
            for file in os.listdir(path):
                img = cv2.imread(os.path.join(path, file))
                img = cv.resize(img ,(224,224,3)) # need to change to the size vgg-16 expects
                img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                print("img dim", img.shape)
                imgs.append(img)
                labels.append([extract_label(file)])
                
    return imgs, labels

In [ ]:
# transfer learning modifications to the model

from tensorflow.keras.applications.vgg16 import VGG16
from keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

model = Sequential()

# pretrained VGG16 model architecture on ImageNet
base_model = VGG16(weights='imagenet', include_top=False)
model.add(base_model)

for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# for layer in base_model.layers:
#     layer.trainable = False

# add other layers
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation='sigmoid')) 
# for binary; it is model.add(Dense(3, activation='softmax')) for the multi-class version


In [ ]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])